In [41]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re
import dask.dataframe as dd


Overwriting testutility.py


In [22]:
%%time
data = pd.read_csv("data2017_2020.csv")
data

Wall time: 186 ms


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
0,00000001_000.png,Cardiomegaly,0,1,57,M,PA,2682,2749,0.143,0.143
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168
3,00000002_000.png,No Finding,0,2,80,M,PA,2500,2048,0.171,0.171
4,00000003_001.png,Hernia,0,3,74,F,PA,2500,2048,0.168,0.168
...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,38,M,PA,2048,2500,0.168,0.168
112116,00030802_000.png,No Finding,0,30802,28,M,PA,2048,2500,0.168,0.168
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168
112118,00030804_000.png,No Finding,0,30804,29,F,PA,2048,2500,0.168,0.168


In [29]:
%%time
data_dask = dd.read_csv("data2017_2020.csv")
data_dask.tail()

Wall time: 194 ms


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
112115,00030801_001.png,Mass|Pneumonia,1,30801,38,M,PA,2048,2500,0.168,0.168
112116,00030802_000.png,No Finding,0,30802,28,M,PA,2048,2500,0.168,0.168
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168
112118,00030804_000.png,No Finding,0,30804,29,F,PA,2048,2500,0.168,0.168
112119,00030805_000.png,No Finding,0,30805,26,M,PA,2048,2500,0.171,0.171


In [31]:
%%time
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Data loading") \
    .getOrCreate()

parking = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("data2017_2020.csv")
parking.show(5)

+----------------+--------------------+-----------+----------+-----------+--------------+-------------+-------------------+-------+---------------------------+-------------------+
|     Image Index|      Finding Labels|Follow-up #|Patient ID|Patient Age|Patient Gender|View Position|OriginalImage[Width|Height]|OriginalImagePixelSpacing[x|                 y]|
+----------------+--------------------+-----------+----------+-----------+--------------+-------------+-------------------+-------+---------------------------+-------------------+
|00000001_000.png|        Cardiomegaly|          0|         1|         57|             M|           PA|               2682|   2749|        0.14300000000000002|0.14300000000000002|
|00000001_001.png|Cardiomegaly|Emph...|          1|         1|         58|             M|           PA|               2894|   2729|        0.14300000000000002|0.14300000000000002|
|00000001_002.png|Cardiomegaly|Effu...|          2|         1|         58|             M|           

### Read YAML

In [33]:

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


### Write YAML

In [38]:
%%writefile file.yaml
file_type: csv
dataset_name: data2017_2020
file_name: NIH Chest X-ray dataset
table_name: CXR8
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 


Overwriting file.yaml


In [40]:
from box import Box
with open("file.yaml", "r") as ymlfile:
    cfg = Box(yaml.safe_load(ymlfile))

ModuleNotFoundError: No module named 'box'